# Chemical equilibrium with fixed pH


In this tutorial we demonstrate how chemical equilibrium calculations with
fixed pH value can be accomplished in Reaktoro.

We'll consider a relatively simple chemical description of an aqueous solution.
This will permit us to describe the concepts and implications of such pH
constraint on the computed equilibrium state.

Let's then create a chemical system with just a single aqueous phase containing
some selected species (instead of all possible species in the database with a
given list of elements). For this phase, we use a Debye–Hückel activity model.
See code below:

In [54]:
import reaktoro as rkt
import numpy as np

db = rkt.SupcrtDatabase("supcrtbl")

solution = rkt.AqueousPhase("H2O(aq) H+ OH- HCO3- CO2(aq) CO3-2")
solution.setActivityModel(rkt.ActivityModelDebyeHuckel())

system = rkt.ChemicalSystem(db, solution)

We want to perform a chemical equilibrium calculation in which the following are constrained:

* temperature;
* pressure; and
* pH;

Let's create a dedicated and optimized chemical equilibrium solver to deal with
these types of problems. First, we create an object of class
`EquilibriumSpecs`, which we use to provide the equilibrium specifications for
our solver later on:

In [55]:
specs = rkt.EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.pH()

Once we have created this specifications object, we can now create our
equilibrium solver:

In [56]:
solver = rkt.EquilibriumSolver(specs)

Let's now create an initial chemical state for our chemical system:

In [57]:
state = rkt.ChemicalState(system)
state.temperature(30.0, "celsius")
state.pressure(1.0, "bar")
state.set("H2O(aq)", 1.0, "kg")
state.set("CO2(aq)", 0.1, "mol")

Clearly, this state is not in chemical equilibrium. Before we equilibrate it
with our equilibrium solver, let's create a copy of the vector of element
amounts in this state, which we call `b0` below. We do this because we will
compare the amounts of each element before and after equilibration, in which
temperature, pressure and pH are prescribed.

In [58]:
b0 = state.elementAmounts().asarray()

Let's now define the temperature, pressure and pH conditions we want to impose
at equilibrium for this aqueous solution currently in disequilibrium. We do
this by using an object of class `EquilibriumConditions`:

In [59]:
conditions = rkt.EquilibriumConditions(specs)
conditions.temperature(50.0, "celsius")
conditions.pressure(10.0, "bar")
conditions.pH(2.0)

We have everything we need now to perform the equilibrium calculation:

* an equilibrium solver (the `solver` object of class `EquilibriumSolver`);
* a chemical state to equilibrate (the `state` object of class `ChemicalState`); and
* the conditions we are imposing for this equilibrium state (the `conditions`
  object of class `EquilibriumConditions`).

The code below will perform the equilibrium calculation and when it is
finished, our `state` object will correspond to a chemical equilibrium state:

In [60]:
result = solver.solve(state, conditions)

It's always advisable to verify if the calculation succeeded:

In [61]:
print("Successful computation!" if result.optima.succeeded else "Computation has failed!")

Successful computation!


Let's check the computed chemical equilibrium state: 

In [62]:
print(state)

+-----------------+-------------+------+
| Property        |       Value | Unit |
+-----------------+-------------+------+
| Temperature     |      323.15 |    K |
| Pressure        |       1e+06 |   Pa |
| Element Amount: |             |      |
| :: H            |     111.028 |  mol |
| :: C            |         0.1 |  mol |
| :: O            |     55.7084 |  mol |
| Species Amount: |             |      |
| :: H2O(aq)      |     55.5084 |  mol |
| :: H+           |   0.0106512 |  mol |
| :: OH-          | 5.78215e-12 |  mol |
| :: HCO3-        | 5.81221e-06 |  mol |
| :: CO2(aq)      |   0.0999942 |  mol |
| :: CO3-2        | 4.88085e-14 |  mol |
+-----------------+-------------+------+


Well, there is no pH in this table. This is because `ChemicalState` objects
only store temperature, pressure, and species amounts. 

What we need now is an object of class `AqueousProps`. This class was designed
for a more convenient inspection of thermodynamic and chemical properties
related to aqueous solutions. Let's create an `AqueousProps` object and print
it:

In [63]:
aprops = rkt.AqueousProps(state)
print(aprops)

+--------------------------+-------------+-------+
| Property                 |       Value |  Unit |
+--------------------------+-------------+-------+
| Temperature              |      323.15 |     K |
| Pressure                 |       1e+06 |    Pa |
| Ionic Strength (Effect.) |  0.00532849 | molal |
| Ionic Strength (Stoich.) |  0.00532849 | molal |
| pH                       |           2 |       |
| pE                       |    -1.34155 |       |
| Eh                       |  -0.0860201 |     V |
| Element Molality:        |             |       |
| :: C                     |         0.1 | molal |
| Species Molality:        |             |       |
| :: H+                    |   0.0106512 | molal |
| :: OH-                   | 5.78216e-12 | molal |
| :: HCO3-                 | 5.81222e-06 | molal |
| :: CO2(aq)               |   0.0999943 | molal |
| :: CO3-2                 | 4.88086e-14 | molal |
+--------------------------+-------------+-------+


Great! We can now see that pH is exactly 2, as we enforced, and temperature and
pressure are 50 °C and 10 bar (but displayed in the table as 323.15 K and
10<sup>6</sup> Pa respectively).

Let's now compare the amounts of elements at equilibrium state with that at the
initial state. 

In [66]:
b = state.elementAmounts().asarray()

for i, element in enumerate(system.elements()):
    print(f"Δb[{element.symbol()}] = {b[i] - b0[i]}")

Δb[H] = 0.010645355059665462
Δb[C] = 1.4155343563970746e-15
Δb[O] = 7.105427357601002e-15


While both elements C and O are numerically conserved (i.e., their final and
initial amounts are equal up to machine precision, which is approximately
1.11e-16), the same cannot be said about element H.

This is because H should not be expected to be preserved in this particular
equilibrium problem, because our chemical system is implicitly open to
H<sup>+</sup>. It is not thermodynamically possible in general to impose,
simultaneously, temperature, pressure, pH and conservation of all chemical
elements and electric charge. 

To impose pH, we need to discard either temperature or pressure constraints, or
make the system open to a substance. Because pH is not strongly sensitive to
temperature and pressure changes, equilibrium calculations with fixed pH are
usually done assuming that an unknown amount of a specified titrant (acid or
base) is added into (or removed from) the system.

```{note}
The current implicit behavior in Reaktoro in which pH constraint presumes that
the system is open to H<sup>+</sup> will be removed in the near future. We will
instead have a more explicit approach in which pH constraints are specified
together with a titrant for which the system is open to. 

Next time you check this tutorial, you may find that `specs.pH("H+")` is needed
instead of `specs.pH()` to achieve identical results we have now. 

This will more clearly specify that the system is open to H<sup>+</sup>. With
this planned change, we'll even be able to specify other titrants, such as HCl,
NaCl, CO2, etc., using the syntax `specs.pH("HCl")`.
```

With this we conclude this tutorial. Stay tuned for an upcoming revision!